# PyironFlow demo

Uncomment the following line if you have edited the .js files

In [1]:
# !npx esbuild js/widget.jsx --minify --format=esm --bundle --outdir=static

In [2]:
%config IPCompleter.evaluation='unsafe'

import sys
from pathlib import Path
sys.path.insert(0, str(Path(Path.cwd()).parent) + '/pyiron_nodes')

In [3]:
from pyiron_workflow import Workflow   

import pyiron_nodes as pn

wf = Workflow('compute_elastic_constants')
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.bulk('Pb', cubic=True)
wf.elastic = pn.atomistic.property.elastic.elastic_constants(structure=wf.bulk, engine=wf.engine) #, parameters=parameters)

# wf.elastic.pull()

In [4]:
from python.pyironflow import PyironFlow

pf = PyironFlow([])
pf.gui

### Some extra features

#### Get the nodes from the gui

In [5]:
import json
json.loads(pf.wf_widgets[0].gui.nodes);

#### Show the workflow widget from tab_0

In [6]:
pf.wf_widgets[0].gui

ReactFlowWidget()

#### Get the current workflow from the gui and visualize it (test completeness of switching between graphical and programmatic representation)

In [7]:
wf = pf.get_workflow()

pf = PyironFlow([wf])
pf.gui

### Built Lammps workflow from scratch

In [8]:
from pyiron_workflow import Workflow   
import pyiron_nodes as pn


In [9]:
wf = Workflow('Lammps')
wf.structure = pn.atomistic.structure.build.bulk('Al', cubic=True)
wf.repeat = pn.atomistic.structure.transform.repeat(structure=wf.structure, repeat_scalar=3)

wf.calculator = pn.lammps.CalcMD() # temperature=300, n_ionic_steps=10_000)
wf.potential = pn.atomistic.engine.lammps.Potential(
    structure=wf.structure, name='1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1'
)

wf.init_lammps = pn.atomistic.engine.lammps.InitLammps(
        structure=wf.repeat,
        potential=wf.potential,
        calculator=wf.calculator,
        working_directory="test2",
    )

wf.shell = pn.atomistic.engine.lammps.Shell(
        # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
        working_directory=wf.init_lammps,
    )

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(
    log_file=wf.shell.outputs.log
)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode='md',
)

wf.get_energy_pot = pn.atomistic.engine.lammps.GetEnergyPot(generic=wf.Collect)

out = wf.run()

energies_pot:  [-362.87999968 -358.5129855  -358.4579939  -358.31296039 -358.55784106
 -359.02478623 -358.31887295 -358.83137511 -358.65856012 -358.41650426
 -358.63303643]


In [10]:
wf.get_energy_pot.pull()[0:-1]

array([-362.87999968, -358.5129855 , -358.4579939 , -358.31296039,
       -358.55784106, -359.02478623, -358.31887295, -358.83137511,
       -358.65856012])

In [11]:
from python.pyironflow import PyironFlow

pf = PyironFlow([wf])
pf.gui

In [12]:
from python.pyironflow import PyironFlow

pf = PyironFlow([])
pf.gui

In [13]:
wf = Workflow('Lammps')
wf.structure = pn.atomistic.structure.build.bulk('Al', cubic=True)
wf.repeat = pn.atomistic.structure.transform.repeat(structure=wf.structure, repeat_scalar=3)

wf.calculator = pn.lammps.CalcMD()
wf.code = pn.atomistic.engine.lammps.Code(structure=wf.repeat, working_dir='test2', calculator=wf.calculator)

Lammps:  structure (UserInput):
Inputs ['user_input']
OutputsWithInjection ['user_input']
InputSignals ['run', 'accumulate_and_run']
OutputSignals ['ran']


In [14]:
wf.run()

TypeError: Unexpected parameters type                 Parameter                                              Value  \
0                   units                                              metal   
1               dimension                                                  3   
2                boundary                                              p p p   
3              atom_style                                             atomic   
4               read_data                                      structure.inp   
5                 include                                      potential.inp   
6          fix___ensemble                       all nvt temp 300.0 300.0 0.1   
7     variable___dumptime                                         equal 100    
8   variable___thermotime                                         equal 100    
9                timestep                                              0.001   
10               velocity               all create 600.0 10298 dist gaussian   
11